In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
from statsmodels.stats.multitest import multipletests

import pandas as pd
from tqdm import tqdm as tqdm

import plotly.express as px
import plotly.graph_objects as go

from scipy import stats



In [2]:
pcawg_additional_batch = [
    "ANKRD53",
    "MYB",
]

pcawg_first_batch = [
    "EGR1", "TRMT10C", "SECISBP2", 
    "RALY", "ZC3H10", 
    "CDC20", "PRKCB"
]

pcawg_second_batch = [
    "MAP2K4", "TBX3", "FBXW7", 
    "MAP1S", "MECOM", "FAM83B", "TFEB"
]

pcawg_genes = pcawg_additional_batch + pcawg_first_batch + pcawg_second_batch

In [8]:
data = pd.read_excel("./supplement/SupplementalTable9_luciferase_assay_percentage_of_regulation_results.xlsx", skiprows=1)

data = data[[col for col in list(data.columns) if col in pcawg_genes]]

In [9]:
data

,EGR1,TRMT10C,ZC3H10,SECISBP2,CDC20,RALY,PRKCB,MAP2K4,TBX3,FBXW7,MAP1S,MECOM,FAM83B,TFEB,MYB
0,55.327002,130.766596,-24.375915,34.312209,94.847966,94.617600,50.771472,-51.224685,-97.012982,8.586133,-61.840825,-37.970300,-57.810895,42.468618,14.309724
1,31.220727,166.928547,-42.583962,-3.436165,72.219366,159.150859,69.615632,-23.032898,-70.297272,-34.866946,-38.719930,-23.844694,21.021898,54.052072,75.584404
2,92.731010,8.607706,-24.522597,4.172270,136.206475,37.444499,107.170089,-3.663856,-44.391790,3.558465,-22.477861,-10.885964,3.753059,55.933174,59.199466
3,34.327014,-15.233936,NaN,NaN,559.366393,29.604207,18.013577,-5.242065,-65.063343,5.039517,-30.288488,-66.383766,6.162148,25.950080,102.760917
4,355.813041,-18.990553,NaN,NaN,484.658707,90.365988,16.475494,NaN,NaN,NaN,NaN,NaN,NaN,73.238115,143.076161
5,220.742466,NaN,NaN,NaN,556.506122,84.400519,6.748845,NaN,NaN,NaN,NaN,NaN,NaN,156.583531,165.440355
6,230.170140,NaN,NaN,NaN,105.619240,13.044525,14.561382,NaN,NaN,NaN,NaN,NaN,NaN,180.663840,242.981033
7,79.131973,NaN,NaN,NaN,171.292184,3.723505,24.355095,NaN,NaN,NaN,NaN,NaN,NaN,157.822582,134.717610
8,275.725056,NaN,NaN,NaN,42.928454,53.679108,26.027870,NaN,NaN,NaN,NaN,NaN,NaN,41.488731,102.568368
9,216.516594,NaN,NaN,NaN,NaN,21.847661,28.524796,NaN,NaN,NaN,NaN,NaN,NaN,65.124837,158.345001


In [22]:
# Rename the genes to have the proper notation.
results = pd.read_excel(
    "./supplement/SupplementalTable1_pSNV_acronyms_and_positions.xlsx",
    skiprows=1,
)

gene_naming = {}
for idx, row in results.iterrows():
    gene = row["Associated Gene"]
    pos = str(row["Position"])[-3:]
    ref = row["Reference Nucleotide"]
    alt = row["Alternative Nucleotide"]
    
    gene_naming[gene] = f"{gene}<sub>{ref}{pos}{alt}</sub>"


data = data.rename(gene_naming, axis=1)

In [ ]:
data

,EGR1<sub>C049T</sub>,TRMT10C<sub>T976G</sub>,ZC3H10<sub>G026C</sub>,SECISBP2<sub>G357A</sub>,CDC20<sub>G528A</sub>,RALY<sub>C916T</sub>,PRKCB<sub>C963T</sub>,MAP2K4<sub>G099A</sub>,TBX3<sub>G045A</sub>,FBXW7<sub>G413A</sub>,MAP1S<sub>C242T</sub>,MECOM<sub>C249T</sub>,FAM83B<sub>C199T</sub>,TFEB<sub>T989G</sub>,MYB<sub>C964A</sub>
0,55.327002,130.766596,-24.375915,34.312209,94.847966,94.617600,50.771472,-51.224685,-97.012982,8.586133,-61.840825,-37.970300,-57.810895,42.468618,14.309724
1,31.220727,166.928547,-42.583962,-3.436165,72.219366,159.150859,69.615632,-23.032898,-70.297272,-34.866946,-38.719930,-23.844694,21.021898,54.052072,75.584404
2,92.731010,8.607706,-24.522597,4.172270,136.206475,37.444499,107.170089,-3.663856,-44.391790,3.558465,-22.477861,-10.885964,3.753059,55.933174,59.199466
3,34.327014,-15.233936,NaN,NaN,559.366393,29.604207,18.013577,-5.242065,-65.063343,5.039517,-30.288488,-66.383766,6.162148,25.950080,102.760917
4,355.813041,-18.990553,NaN,NaN,484.658707,90.365988,16.475494,NaN,NaN,NaN,NaN,NaN,NaN,73.238115,143.076161
5,220.742466,NaN,NaN,NaN,556.506122,84.400519,6.748845,NaN,NaN,NaN,NaN,NaN,NaN,156.583531,165.440355
6,230.170140,NaN,NaN,NaN,105.619240,13.044525,14.561382,NaN,NaN,NaN,NaN,NaN,NaN,180.663840,242.981033
7,79.131973,NaN,NaN,NaN,171.292184,3.723505,24.355095,NaN,NaN,NaN,NaN,NaN,NaN,157.822582,134.717610
8,275.725056,NaN,NaN,NaN,42.928454,53.679108,26.027870,NaN,NaN,NaN,NaN,NaN,NaN,41.488731,102.568368
9,216.516594,NaN,NaN,NaN,NaN,21.847661,28.524796,NaN,NaN,NaN,NaN,NaN,NaN,65.124837,158.345001


In [25]:
def _caclulate_effect_size(values: list):
    """Cohen's d (one-sided)
    
    d = ((x_mean) - (mu)) / s
    
    x_mean = sample mean
    mu = mean under the null hypothesis, which is 0 (i.e. no difference)
    s = sample standard deviation

    Parameters
    ----------
    values : list
        List of replicate values.
    """
    x_mean = np.mean(values)
    mu = 0
    s = np.std(values, ddof=1) # Use 1 to denote sample std rather than population
    
    return (x_mean - mu) / s
    

In [26]:
pval_dataframe = pd.DataFrame()

for idx, gene_name in enumerate(data.columns):
    values = [i for i in data[gene_name] if not np.isnan(i)]
    pval = ttest_1samp(values, popmean=0, alternative="greater").pvalue
    t_stat = ttest_1samp(values, popmean=0, alternative="greater").statistic
    pval_dataframe.loc[idx, "gene_name"] = gene_name
    pval_dataframe.loc[idx, "num_replicates"] = len(values)
    pval_dataframe.loc[idx, "p-value"] = pval
    pval_dataframe.loc[idx, "p-value_significant"] = pval <= 0.05
    pval_dataframe.loc[idx, "test_statistic"] = t_stat

    pval_dataframe.loc[idx, "effect_size"] = _caclulate_effect_size(values)
    


In [27]:
adjusted_results = multipletests(pval_dataframe['p-value'], method='fdr_bh')  # Benjamini-Hochberg
pval_dataframe["q-value"] = adjusted_results[1]
pval_dataframe["q-value_significant"] = adjusted_results[0]


## Figures

In [29]:
replicate_dataframe = pd.DataFrame()

counter = 0
for column in data.columns:
    for idx, val in enumerate([i for i in list(data[column]) if not np.isnan(i)]):
        replicate_dataframe.loc[counter, "gene_names"] = column
        replicate_dataframe.loc[counter, "percentage_of_upregulation"] = val
        replicate_dataframe.loc[counter, "replicate"] = f"Replicate {idx+1}"
        counter += 1
        
replicate_dataframe

,gene_names,percentage_of_upregulation,replicate
0,EGR1<sub>C049T</sub>,55.327002,Replicate 1
1,EGR1<sub>C049T</sub>,31.220727,Replicate 2
2,EGR1<sub>C049T</sub>,92.731010,Replicate 3
3,EGR1<sub>C049T</sub>,34.327014,Replicate 4
4,EGR1<sub>C049T</sub>,355.813041,Replicate 5
...,...,...,...
94,MYB<sub>C964A</sub>,165.440355,Replicate 6
95,MYB<sub>C964A</sub>,242.981033,Replicate 7
96,MYB<sub>C964A</sub>,134.717610,Replicate 8
97,MYB<sub>C964A</sub>,102.568368,Replicate 9


In [ ]:

def _plot_percentage_upregulation_v1(
    dataframe: pd.DataFrame,
    title_text: str="Percentage Upregulation for Chosen Candidates",
    fdr: bool=True,
    only_positives: bool=True
):
    if only_positives:
        positives_genes = list(pval_dataframe[(pval_dataframe["p-value_significant_after_fdr_bh"] == True)]["gene_name"])
        dataframe = dataframe[dataframe["gene_names"].isin(positives_genes)].reset_index(drop=False)
    
    fig = go.Figure()
    
    # Add all replicates.
    fig = px.strip(
        dataframe, 
        x="gene_names", 
        y="percentage_of_upregulation",
    )
    

    fig.update_xaxes(
        showgrid=True, 
        ticks="outside", 
        tickson="boundaries",
        tickangle=45
    )
    
    fig.update_traces(jitter=0.2)
    fig.update_traces({'marker': {'size': 10, 'color': 'lightgrey', "opacity": 1}})

    # Add average value.
    averages = []
    for idx, gene in enumerate(dataframe.gene_names.unique()):
        avg = np.mean(dataframe[dataframe["gene_names"] == gene]["percentage_of_upregulation"])
        averages.append((gene, avg))
        fig.add_trace(
            go.Scatter(
                x = [gene],
                y = [avg],
                mode="markers",
                marker=dict(
                    color='#e8926c',
                    size=20,
                    line=dict(
                        color='black',
                        width=1
                    ),
                    symbol=17
                ),
                name="Average",
                showlegend=True if idx==0 else False
            )
        )
        
    fig.update_yaxes(
        title = "Percentage of Upregulation",
        # dtick = 20
    )

    fig.update_xaxes(
        title="Name of Validated Gene",
    )

    # Add p-values.
    pvalue_dict = {}
    for idx, gene in enumerate(dataframe.gene_names.unique()):
        percentage_upregulated = [value for value in list(dataframe[dataframe["gene_names"] == gene]["percentage_of_upregulation"]) if not np.isnan(value)]
        
        if not fdr:
            pval = stats.ttest_1samp(percentage_upregulated, popmean=0, alternative="greater").pvalue
        else:
            pval = pval_dataframe[pval_dataframe["gene_name"] == gene]["q-value"].iloc[0]
            
        text = np.round(float(pval), 3)

        pvalue_dict[gene] = f"q-value={text}"

    # Order by average percentage upregulation.
    order = sorted(
        averages,
        key=lambda x: x[1],
        reverse=True
    )
    
    # Order alphabetically.
    order = sorted(
        averages,
        key=lambda x: x[0],
    )

    for idx, gene_and_val in enumerate(order):
        gene, val = gene_and_val
        fig.add_annotation(
            dict(
                font=dict(color="black",size=12),
                x=idx,
                y=1.07,
                showarrow=False,
                text=pvalue_dict[gene],
                textangle=0,
                xref="x",
                yref="paper"
            )
        )

    fig.update_xaxes(
        categoryorder="array",
        categoryarray=[gene_name for gene_name, perc in order]
    )

    # Add horizontal line at 0.
    # fig.add_hline(
    #     y=0, line_dash="dot", opacity=0.8,
    #     annotation_text="No regulation", 
    #     annotation_position="top right"
    # )

    fig.update_layout(
        title_text=title_text
    )

    fig.update_layout(
        yaxis = {
            "ticksuffix": "%"
        }
    )
    
    fig.update_layout(
        plot_bgcolor='white'
    )

    fig.update_xaxes(
        mirror=True,
        ticks='inside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        linewidth=1
    )

    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        linewidth=1
    )
    
    return fig, order

In [34]:
def _plot_percentage_upregulation_v2(
    dataframe: pd.DataFrame,
    title_text: str="Percentage Upregulation for Chosen Candidates",
    fdr: bool=True,
    only_positives: bool=True
):
    """
    Generates a scatter plot of percentage upregulation for validated genes.

    Args:
        dataframe (pd.DataFrame): Input data with gene names and percentage upregulation.
        title_text (str): Title of the plot.
        fdr (bool): Whether to use False Discovery Rate adjusted p-values.
        only_positives (bool): Whether to filter only significant genes.

    Returns:
        fig (go.Figure): Plotly figure object.
        order (list): Sorted list of genes based on upregulation percentage.
    """
    
    # Filter only significant genes
    if only_positives:
        significant_genes = pval_dataframe.loc[pval_dataframe["p-value_significant_after_fdr_bh"], "gene_name"]
        dataframe = dataframe[dataframe["gene_names"].isin(significant_genes)].reset_index(drop=True)

    fig = go.Figure()

    # Scatter plot of all replicates
    fig = px.strip(
        dataframe, 
        x="gene_names", 
        y="percentage_of_upregulation",
    )

    fig.update_xaxes(
        showgrid=True, 
        ticks="outside", 
        tickson="boundaries",
        tickangle=45
    )

    fig.update_traces(jitter=0.2)
    fig.update_traces({'marker': {'size': 10, 'color': 'lightgrey', "opacity": 1}})

    # Compute averages for each gene
    averages = {
        gene: np.mean(dataframe[dataframe["gene_names"] == gene]["percentage_of_upregulation"])
        for gene in dataframe["gene_names"].unique()
    }

    # Add average markers
    for idx, (gene, avg) in enumerate(averages.items()):
        fig.add_trace(
            go.Scatter(
                x=[gene],
                y=[avg],
                mode="markers",
                marker=dict(
                    color='#e8926c',
                    size=20,
                    line=dict(color='black', width=1),
                    symbol=17
                ),
                name="Average",
                showlegend=(idx == 0)
            )
        )

    # Compute p-values
    pvalue_dict = {}
    for gene in dataframe["gene_names"].unique():
        values = dataframe.loc[dataframe["gene_names"] == gene, "percentage_of_upregulation"].dropna()

        pval = (
            stats.ttest_1samp(values, popmean=0, alternative="greater").pvalue
            if not fdr else
            pval_dataframe.loc[pval_dataframe["gene_name"] == gene, "q-value"].iloc[0]
        )

        pvalue_dict[gene] = f"q-value=<br>{np.round(float(pval), 3)}"

    # Order genes: first by avg percentage upregulation (descending), then alphabetically
    order = sorted(averages.items(), key=lambda x: (-x[1], x[0]))

    # Add annotations for p-values
    for idx, (gene, _) in enumerate(order):
        fig.add_annotation(
            font=dict(color="black", size=12),
            x=idx,
            y=1.13,
            showarrow=False,
            text=pvalue_dict[gene],
            textangle=0,
            xref="x",
            yref="paper"
        )

    # Update x-axis order
    fig.update_xaxes(
        categoryorder="array",
        categoryarray=[gene for gene, _ in order]
    )

    # Update layout and styling
    fig.update_layout(
        title_text=title_text,
        plot_bgcolor='white',
        xaxis=dict(
            mirror=True,
            ticks='inside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
            linewidth=1
        ),
        yaxis=dict(
            ticksuffix = "%",
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
            linewidth=1
        )
    )
    
    return fig, order

In [35]:
# Plot all candidates.
all_candidates_plot, order = _plot_percentage_upregulation_v2(
    dataframe=replicate_dataframe,
    title_text=None,
    only_positives=False
)
all_candidates_plot.update_layout(
    width=1400
)
all_candidates_plot.show()